In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 48.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Carga el dataset desde un archivo CSV u otra fuente
# Por ejemplo, si tienes un archivo CSV llamado "data.csv" con dos columnas "comment" y "medication"
data = pd.read_csv('/content/drive/MyDrive/Datas/comentarios_medicacion.csv')
display(data)

,body,created_datetime,Cleaned Body,Tiene_Medicacion,Medicación
0,potassium is used as the thing that stops your...,2009-06-01 00:07:50,potassium is used as the thing that stops your...,1,adderall
1,I've love a link to anything about this. \n\n...,2009-06-11 20:25:36,i ve love a link to anything about this i have...,1,adderall
2,Wow... There was a interruption in the distrib...,2011-06-16 23:38:16,wow there was a interruption in the distributi...,1,ritalin
3,Which country are you in? \n\nTry to see a pri...,2011-06-30 18:10:52,which country are you in try to see a private ...,1,dexmethylphenidate
4,"What the hell, your GP can't get you in for a ...",2011-06-30 22:00:01,what the hell your gp can t get you in for a m...,1,adderall
...,...,...,...,...,...
272327,Yeah that would be me too. How I did in classe...,2021-05-03 23:23:45,yeah that would be me too how i did in classes...,1,concerta
272328,"What a horrible, disturbing way to treat a pat...",2021-05-03 23:25:05,what a horrible disturbing way to treat a pati...,1,atomoxetine
272329,"If you don't mind, did the wellbutrin help wit...",2021-05-03 23:27:01,if you don t mind did the wellbutrin help with...,1,wellbutrin
272330,Well it's not an appetite problem and it's no ...,2021-05-03 23:30:21,well it s not an appetite problem and it s no ...,1,dexmethylphenidate


In [ ]:
data['body'] = data['body'] + ' ' + data['Medicación']

In [ ]:
import re
import string

In [ ]:
def  clean_text(text):
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"thee", "you", text)
    #text = re.sub(r"r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("(\W)"," ",text)
    #text = re.sub('S*dS*s*','', text)
    text = re.sub('[^A-Za-z]+', ' ', text)
    text =  text.lower()
    return text

In [ ]:
data['Cleaned Body'] = data['body'].apply(lambda body: clean_text(body))

# Seleccionar y mostrar las dos variables
columna_seleccionada1 = data['body']
columna_seleccionada2 = data['Cleaned Body']

print(columna_seleccionada1)
print(columna_seleccionada2)

0         potassium is used as the thing that stops your...
1         I've love a link to anything about this.  \n\n...
2         Wow... There was a interruption in the distrib...
3         Which country are you in? \n\nTry to see a pri...
4         What the hell, your GP can't get you in for a ...
                                ...                        
272327    Yeah that would be me too. How I did in classe...
272328    What a horrible, disturbing way to treat a pat...
272329    If you don't mind, did the wellbutrin help wit...
272330    Well it's not an appetite problem and it's no ...
272331    That's awesome you were able to do that! Did y...
Name: body, Length: 272332, dtype: object
0         potassium is used as the thing that stops your...
1         i have love a link to anything about this i ha...
2         wow there was a interruption in the distributi...
3         which country are you in try to see a private ...
4         what the hell your gp cannot get you in for a ..

In [ ]:
medicacion = pd.DataFrame(data['Medicación'])
medicacion.drop_duplicates(inplace=True)
medicacion = medicacion.sort_values('Medicación')
medicacion['medicacion_id'] = medicacion.reset_index().index + 1
display(medicacion)

,Medicación,medicacion_id
280,acids,1
0,adderall,2
58454,adzenys,3
14,amphetamine,4
133,aptensio,5
93,armodafinil,6
259,atomoxetine,7
184,bupropion,8
749,clonidine,9
6,concerta,10


In [ ]:
data=data.merge(medicacion[['Medicación', 'medicacion_id']], on='Medicación', how='left')
display(data)

,body,created_datetime,Cleaned Body,Tiene_Medicacion,Medicación,medicacion_id
0,potassium is used as the thing that stops your...,2009-06-01 00:07:50,potassium is used as the thing that stops your...,1,adderall,2
1,I've love a link to anything about this. \n\n...,2009-06-11 20:25:36,i have love a link to anything about this i ha...,1,adderall,2
2,Wow... There was a interruption in the distrib...,2011-06-16 23:38:16,wow there was a interruption in the distributi...,1,ritalin,38
3,Which country are you in? \n\nTry to see a pri...,2011-06-30 18:10:52,which country are you in try to see a private ...,1,dexmethylphenidate,16
4,"What the hell, your GP can't get you in for a ...",2011-06-30 22:00:01,what the hell your gp cannot get you in for a ...,1,adderall,2
...,...,...,...,...,...,...
272327,Yeah that would be me too. How I did in classe...,2021-05-03 23:23:45,yeah that would be me too how i did in classes...,1,concerta,10
272328,"What a horrible, disturbing way to treat a pat...",2021-05-03 23:25:05,what a horrible disturbing way to treat a pati...,1,atomoxetine,7
272329,"If you don't mind, did the wellbutrin help wit...",2021-05-03 23:27:01,if you do not mind did the wellbutrin help wit...,1,wellbutrin,41
272330,Well it's not an appetite problem and it's no ...,2021-05-03 23:30:21,well it is not an appetite problem and it is n...,1,dexmethylphenidate,16


In [ ]:
# Preprocesamiento de los comentarios
comments = data['Cleaned Body'].values


In [ ]:
# Tokenización y vectorización utilizando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(comments)


In [ ]:
from sklearn.cluster import KMeans
 # Número de clusters que deseas obtener (ajustar según la cantidad de medicamentos que quieras identificar)
n_clusters = 41

# Aplicar K-Means para clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=41, random_state=42)

In [ ]:
print(len(X.indices))

10615395


In [ ]:
# Obtener los índices de los centroides de cada cluster
# Crear un diccionario para almacenar los datos de cada cluster
cluster_data = {}

# Obtener los índices de los centroides de cada cluster
cluster_centers = kmeans.cluster_centers_.argsort()[:, ::-1]

# Obtener las palabras más representativas de cada cluster
terms = vectorizer.get_feature_names_out()

for i in range(n_clusters):
    # Crear un arreglo para almacenar las palabras de cada cluster
    words_in_cluster = []

    print(f"Cluster {i+1}:")
    for ind in cluster_centers[i, :10]:  # Mostrar las 10 palabras más representativas de cada cluster
        cadena = terms[ind]
        #arreglo = f"[Cluster ]{i+1}:"
        arreglo = cadena.split()
        print(arreglo)



# Ahora, cluster_data contendrá los arreglos de palabras representativas de cada cluster.
# Puedes acceder a los datos de cada cluster utilizando el número del cluster como índice en el diccionario.



Cluster 1:
['intuniv']
['stimulants']
['strattera']
['adhd']
['stimulant']
['guanfacine']
['anxiety']
['norepinephrine']
['blood']
['effects']
Cluster 2:
['just']
['things']
['like']
['time']
['work']
['really']
['meds']
['im']
['people']
['know']
Cluster 3:
['release']
['extended']
['instant']
['adderall']
['concerta']
['ritalin']
['vyvanse']
['mg']
['day']
['hours']
Cluster 4:
['bad']
['adderall']
['effects']
['really']
['just']
['like']
['meds']
['im']
['taking']
['good']
Cluster 5:
['strattera']
['straterra']
['effects']
['like']
['im']
['stimulants']
['just']
['did']
['weeks']
['adhd']
Cluster 6:
['anxiety']
['adderall']
['adhd']
['depression']
['vyvanse']
['meds']
['im']
['like']
['just']
['doctor']
Cluster 7:
['coffee']
['drink']
['adderall']
['vyvanse']
['cup']
['caffeine']
['drinking']
['cups']
['day']
['morning']
Cluster 8:
['rsd']
['dodson']
['rejection']
['sensitivity']
['term']
['solely']
['removed']
['scientific']
['dysregulationg']
['propagated']
Cluster 9:
['im']
['adde

In [ ]:
# Obtener las etiquetas de los clusters para cada registro
cluster_labels = kmeans.labels_
data['cluster_labels'] = cluster_labels
display(data)

,body,created_datetime,Cleaned Body,Tiene_Medicacion,Medicación,medicacion_id,cluster_labels
0,potassium is used as the thing that stops your...,2009-06-01 00:07:50,potassium is used as the thing that stops your...,1,adderall,2,36
1,I've love a link to anything about this. \n\n...,2009-06-11 20:25:36,i have love a link to anything about this i ha...,1,adderall,2,36
2,Wow... There was a interruption in the distrib...,2011-06-16 23:38:16,wow there was a interruption in the distributi...,1,ritalin,38,39
3,Which country are you in? \n\nTry to see a pri...,2011-06-30 18:10:52,which country are you in try to see a private ...,1,dexmethylphenidate,16,19
4,"What the hell, your GP can't get you in for a ...",2011-06-30 22:00:01,what the hell your gp cannot get you in for a ...,1,adderall,2,36
...,...,...,...,...,...,...,...
272327,Yeah that would be me too. How I did in classe...,2021-05-03 23:23:45,yeah that would be me too how i did in classes...,1,concerta,10,13
272328,"What a horrible, disturbing way to treat a pat...",2021-05-03 23:25:05,what a horrible disturbing way to treat a pati...,1,atomoxetine,7,24
272329,"If you don't mind, did the wellbutrin help wit...",2021-05-03 23:27:01,if you do not mind did the wellbutrin help wit...,1,wellbutrin,41,11
272330,Well it's not an appetite problem and it's no ...,2021-05-03 23:30:21,well it is not an appetite problem and it is n...,1,dexmethylphenidate,16,8


In [ ]:
cluster_centers = kmeans.cluster_centers_.argsort()[:, ::-1]
def grupoC(i):
  SS = terms[cluster_centers[i,:40]]
  # Mostrar las 10 palabras más representativas de cada cluster
  return SS


In [ ]:
N = grupoC(36)
print(N)

['adderall' 'just' 'like' 'taking' 'im' 'meds' 'doctor' 'time' 'day' 'did'
 'adhd' 'people' 'really' 'think' 'know' 'does' 'work' 'started' 'effects'
 'years' 'feel' 'try' 'took' 'need' 'got' 'good' 'better' 'different'
 'make' 'help' 'makes' 'medication' 'days' 'way' 'focus' 'mg' 'thing'
 'drug' 'helps' 'actually']


In [ ]:
data['representative_words'] = data['cluster_labels'].apply(grupoC)

In [ ]:
display(data)

,body,created_datetime,Cleaned Body,Tiene_Medicacion,Medicación,medicacion_id,cluster_labels,representative_words
0,potassium is used as the thing that stops your...,2009-06-01 00:07:50,potassium is used as the thing that stops your...,1,adderall,2,36,"[adderall, just, like, taking, im, meds, docto..."
1,I've love a link to anything about this. \n\n...,2009-06-11 20:25:36,i have love a link to anything about this i ha...,1,adderall,2,36,"[adderall, just, like, taking, im, meds, docto..."
2,Wow... There was a interruption in the distrib...,2011-06-16 23:38:16,wow there was a interruption in the distributi...,1,ritalin,38,39,"[ritalin, just, im, like, meds, day, adhd, tak..."
3,Which country are you in? \n\nTry to see a pri...,2011-06-30 18:10:52,which country are you in try to see a private ...,1,dexmethylphenidate,16,19,"[dexmethylphenidate, methylphenidate, concerta..."
4,"What the hell, your GP can't get you in for a ...",2011-06-30 22:00:01,what the hell your gp cannot get you in for a ...,1,adderall,2,36,"[adderall, just, like, taking, im, meds, docto..."
...,...,...,...,...,...,...,...,...
272327,Yeah that would be me too. How I did in classe...,2021-05-03 23:23:45,yeah that would be me too how i did in classes...,1,concerta,10,13,"[concerta, like, just, concept, acids, im, ome..."
272328,"What a horrible, disturbing way to treat a pat...",2021-05-03 23:25:05,what a horrible disturbing way to treat a pati...,1,atomoxetine,7,24,"[medication, adderall, doctor, adhd, vyvanse, ..."
272329,"If you don't mind, did the wellbutrin help wit...",2021-05-03 23:27:01,if you do not mind did the wellbutrin help wit...,1,wellbutrin,41,11,"[wellbutrin, adhd, depression, im, did, just, ..."
272330,Well it's not an appetite problem and it's no ...,2021-05-03 23:30:21,well it is not an appetite problem and it is n...,1,dexmethylphenidate,16,8,"[im, adderall, just, like, vyvanse, really, me..."


In [ ]:
# Especifica la ruta y el nombre de archivo para guardar el archivo Excel
nombre_archivo = 'Comentarios_clustering.csv'

# Guarda el DataFrame en un archivo CSV
data.to_csv(nombre_archivo)

CNN

In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.model_selection import train_test_split


In [ ]:
# Datos de ejemplo

comentarios =  data['Cleaned Body']
etiquetas = data['cluster_labels']




# Convertir los datos a listas de cadenas de texto
comentarios = comentarios.tolist()
etiquetas = etiquetas.tolist()

# Convertir las etiquetas a valores numéricos
etiquetas_num = data['cluster_labels']
# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(comentarios, etiquetas_num, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_train = X_train.tolist
# Tokenización y creación del vocabulario
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Convertir comentarios a secuencias numéricas
X_test = np.array(X_test)
X_test = X_test.tolist()
secuencias_train = tokenizer.texts_to_sequences(X_train)
secuencias_test = tokenizer.texts_to_sequences(X_test)

# Relleno de secuencias para tener la misma longitud
max_length = max([len(seq) for seq in secuencias_train])
X_train_pad = pad_sequences(secuencias_train, maxlen=max_length)
X_test_pad = pad_sequences(secuencias_test, maxlen=max_length)

In [ ]:
# Creación del modelo de CNN
model = Sequential()
model.add(Embedding(vocab_size, 1, input_length=max_length))
model.add(Conv1D(1, 1, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train_pad, y_train, epochs=10, batch_size=16, validation_data=(X_test_pad, y_test))

Epoch 1/10
13617/13617 [==============================] - 38s 3ms/step - loss: -7696150.5000 - accuracy: 0.0121 - val_loss: -31879356.0000 - val_accuracy: 0.0121
Epoch 2/10
13617/13617 [==============================] - 40s 3ms/step - loss: -135735600.0000 - accuracy: 0.0121 - val_loss: -308555232.0000 - val_accuracy: 0.0121
Epoch 3/10
13617/13617 [==============================] - 39s 3ms/step - loss: -695104320.0000 - accuracy: 0.0121 - val_loss: -1233505664.0000 - val_accuracy: 0.0121
Epoch 4/10
13617/13617 [==============================] - 37s 3ms/step - loss: -2179571968.0000 - accuracy: 0.0121 - val_loss: -3389094144.0000 - val_accuracy: 0.0121
Epoch 5/10
13617/13617 [==============================] - 45s 3ms/step - loss: -5261333504.0000 - accuracy: 0.0121 - val_loss: -7534016512.0000 - val_accuracy: 0.0121
Epoch 6/10
13617/13617 [==============================] - 38s 3ms/step - loss: -10775943168.0000 - accuracy: 0.0121 - val_loss: -14585943040.0000 - val_accuracy: 0.0121
Epoc

Hibrido con CNN

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Input

# Supongamos que tienes un DataFrame 'data' con las columnas 'text', 'cluster_label' y 'representative_words'
# También tienes una columna 'representative_words' con las 10 palabras más representativas de cada cluster.



In [ ]:
# Convertir texto a secuencias numéricas usando Tokenizer
max_words = 1000  # Número máximo de palabras en el vocabulario
max_sequence_length = 100  # Longitud máxima de la secuencia
tokenizer = Tokenizer(num_words=max_words)


In [ ]:
# Obtener el número total de clusters
n_clusters

41

In [ ]:
# Obtener todas las palabras representativas de los clusters y convertirlas a cadenas
all_representative_words = data['representative_words'].tolist()
all_representative_words = [' '.join(words) for words in all_representative_words if isinstance(words, list) and all(isinstance(word, str) for word in words)]

# Convertir todas las palabras a minúsculas
all_representative_words = [words.lower() for words in all_representative_words]

tokenizer.fit_on_texts(all_representative_words)
sequences = tokenizer.texts_to_sequences(all_representative_words)
X_text = pad_sequences(sequences, maxlen=max_sequence_length)




ValueError: ignored

In [ ]:
def get_representative_words(cluster_number):
    cluster_data = data[data['cluster_labels'] == cluster_number]
    if len(cluster_data) > 0:
        cluster_words = cluster_data['representative_words'].iloc[0]
        return cluster_words.tolist()  # Convertir a lista de palabras
    else:
        return None


In [ ]:
X_cluster_representations = []
for i in range(1, n_clusters+1):  # +1 para que el rango incluya n_clusters
    cluster_words = get_representative_words(i)
    if cluster_words is not None and len(cluster_words) > 0:
        cluster_sequence = tokenizer.texts_to_sequences([' '.join(cluster_words)])
        X_cluster_representations.append(cluster_sequence[0])

In [ ]:
# Construir el modelo híbrido
embedding_dim = 50
input_text = Input(shape=(max_sequence_length,))
input_cluster = Input(shape=(max_sequence_length,))

embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim)

In [ ]:
text_embedded = embedding_layer(input_text)
cluster_embedded = embedding_layer(input_cluster)

text_conv = Conv1D(128, 5, activation='relu')(text_embedded)
text_pool = GlobalMaxPooling1D()(text_conv)

cluster_conv = Conv1D(128, 5, activation='relu')(cluster_embedded)
cluster_pool = GlobalMaxPooling1D()(cluster_conv)

concatenated = Concatenate()([text_pool, cluster_pool])
output = Dense(n_clusters, activation='softmax')(concatenated)

model = Model(inputs=[input_text, input_cluster], outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Construir la representación numérica de las palabras más representativas de cada cluster
X_cluster_representations = []
cluster_words = all_representative_words # +1 porque los clusters están numerados desde 1
cluster_sequence = tokenizer.texts_to_sequences([' '.join(cluster_words)])
cluster_representation = pad_sequences(cluster_sequence, maxlen=max_sequence_length)
if len(cluster_representation) > 0:  # Verificar que hay palabras representativas válidas
    X_cluster_representations.append(cluster_representation[0])
else:
        # Si no hay palabras representativas, agregar un valor no válido (por ejemplo, ceros)
    X_cluster_representations.append(np.zeros(max_sequence_length))

X_cluster_representations = np.array(X_cluster_representations)

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split

In [ ]:
# Convertir los arrays a matrices de NumPy
X_text = np.array(X_text)
X_cluster_representations = np.array(X_cluster_representations)

# Verificar las formas de los arrays
print(X_text.shape[0])
print(X_cluster_representations.shape[0])

#Asegurarse de que tengan la misma cantidad de muestras
assert X_text.shape[0] == X_cluster_representations.shape[0]
print(X_text.shape[0])
print(X_cluster_representations.shape[0])
# Continuar con el proceso de división de datos
from sklearn.model_selection import train_test_split
X_train_text, X_test_text, X_train_cluster, X_test_cluster, y_train, y_test = train_test_split(
    X_text, X_cluster_representations, data['cluster_labels'], test_size=0.2, random_state=42)


0
1


AssertionError: ignored